### Location Inference

------------

In [1]:
import pandas as pd
import numpy as np
import csv

In [ ]:
def find_city (twt):
    
    for index, row in twt.iterrows():
        
        city=row['place.name']
        if (df['GeographicalName'].str.match(city)).any():
        
            twt['City'][index]=city
            print (city)
        else:
            twt['City'][index]= "dummy"
    #---------------------------------------------------------------------------------
    #We need to fix the Province first, using the twitter dataset. Because there might be multiple cities with the same name (e.g. Jasper).
        
        try:
            province=row['place.full_name'].rsplit(', ', 1)[1] #returns the province in (Calgary, Alberta)

        except IndexError:
            province = row['place.full_name'].rsplit(', ', 1)[-1]
            #print(row['place.full_name'].rsplit(', ', 1)[-1])

        if df['Province'].str.match(province).any():
            twt['Province'][index]=province
            print(province)
            print("----------------------------")
        else:
            twt['Province'][index]= "dummy" 
           
        #In case we could not detect the province using the Twitter's field (but we have the city from that field)    
        if (df[df['GeographicalName']==city]['Province'].any() and twt['Province'][index]== "dummy" ):
            twt['Province'][index]=df[df['GeographicalName']==city]['Province'].values[0] 
            print(df[df['GeographicalName']==city]['Province'].values[0] )   
            print("----------------------------")
        #sometimes there is a , after the city name + some alphabets + Province name     
        
           
    return twt

In [ ]:
def find_city_old (twt):
    twt['City']="dummy"
    twt['Province']="dummy"
    twt['place.name']= twt['place.name'].str.replace('\(', '')
    twt['place.name']= twt['place.name'].str.replace('\)', '')
    for index, row in twt.iterrows():
        #sometimes there is a , after the city name + some alphabets + Province name 
        for i in reversed(range(len(row['place.name'].rsplit(',', 1)[0].split()))):
            
            city= " ".join(row['place.name'].rsplit(',', 1)[0].split()[:i+1])
            print(city)
            if df['GeographicalName'].str.match(city).any():
                twt['City'][index]=city
                
                if (df[df['GeographicalName']==city]['Province'].any()):
                   
                    twt['Province'][index]=df[df['GeographicalName']==city]['Province'].values[0] 
                    print(df[df['GeographicalName']==city]['Province'].values[0])
                break
         
        if (twt['City'][index]== "dummy"):
            try:
                province=row['place.full_name'].rsplit(', ', 1)[1] #return the province in (Calgary, Alberta)
            except IndexError:
                province = row['place.full_name'].rsplit(', ', 1)[-1]
                #print(row['place.full_name'].rsplit(', ', 1)[-1])

            if df['Province'].str.match(province).any():
                twt['Province'][index]=province
            else:
                twt['Province'][index]= "dummy"       
    return twt
            

In [ ]:
def contains_word(s, w): #find the whole word
    return f' {w} ' in f' {s} '



The following function checks if it can extract the city/province names from the vague descriptions. For example, **the beautiful city of Toronto** can be mapped to **Toronto**.


In [ ]:
def find_dummy (twt):
     for index, row in twt.iterrows():
            if (row['City']=="dummy"):
                for i in range(len(cities)):
                    
                    w = cities[i].lower()
                    s = twt['place.full_name'][index].lower()
                    if (f' {w} ' in f' {s}'):
                        print("true")
                    #if (locations[i].lower() in twt['place.full_name'][index].lower()):
                        twt['City'][index]= df[df['GeographicalName']==cities[i]]['GeographicalName'].values[0] 
                        print (df[df['GeographicalName']==cities[i]]['GeographicalName'].values[0] )
                        if (twt['Province'][index]!= "dummy"):
                            if (twt['Province'][index]!= df[df['GeographicalName']==cities[i]]['Province'].values[0]):
                                print("****************************ERROR********************")
                                twt['City'][index]= "dummy" #the found city is wrong, remove the found one!
                        else:
                            twt['Province'][index]=df[df['GeographicalName']==cities[i]]['Province'].values[0]     
                        break

In [ ]:
def find_from_text (twt):
     for index, row in twt.iterrows():
            
            if (row['City']=="dummy"):
                for i in range(len(cities_only)):
                    w = cities_only[i].lower()
                    s = twt['text'][index].lower()
                    if (f' {w} ' in f' {s}'):
                        
                    #if (cities[i].lower() in twt['text'][index].lower()):
                        twt['City'][index]= df[df['GeographicalName']==cities_only[i]]['GeographicalName'].values[0] 
                        print (df[df['GeographicalName']==cities_only[i]]['GeographicalName'].values[0] )
                        twt['Province'][index]=df[df['GeographicalName']==cities_only[i]]['Province'].values[0]    

In [ ]:

def find_from_user (twt):
     for index, row in twt.iterrows():
            if (row['City']=="dummy"):
                for i in range(len(cities_only)):
                    w = cities_only[i].lower()
                    s = twt['user.description'][index].lower()
                    if (f' {w} ' in f' {s}'):
                    #if (cities[i].lower() in twt['text'][index].lower()):
                        twt['City'][index]= df[df['GeographicalName']==cities_only[i]]['GeographicalName'].values[0] 
                        print (df[df['GeographicalName']==cities_only[i]]['GeographicalName'].values[0] )
                        twt['Province'][index]=df[df['GeographicalName']==cities_only[i]]['Province'].values[0]   

In [ ]:
def find_dummy_province(twt):
    prov= df['Province'].unique()
    for index, row in twt.iterrows():
            if (row['Province']=="dummy"):
                for i in range(len(prov)):
                    
                    w = prov[i]
                    s = twt['place.full_name'][index]
                    if (f' {w} ' in f' {s}'):
                        print("true")
                    #if (locations[i].lower() in twt['place.full_name'][index].lower()):
                        twt['Province'][index]=w
                        print (w)   
                        break

In [ ]:
def final_find (twt):
    geo= df['GeographicalName'].unique()
    print(len(geo))
    for index, row in twt.iterrows():
        l_match=''
        if (row['City']=="dummy"):
            for i in range(len(geo)):
                
                w = geo[i]
                
                s = twt['place.name'][index]
                if (f' {w} ' in f' {s}'):
                    print(w)
                    if len(w)>len(l_match): #if there is fraser and fraser valley, we want to catch the longest one!
                        l_match=w
            print(l_match)
                #if (cities[i].lower() in twt['text'][index].lower()):
            if len(l_match)>0:               
                 
                #print (df[df['GeographicalName']==l_match]['GeographicalName'].values[0] )
                if (twt['Province'][index]!= "dummy"):
                        if (twt['Province'][index]!= df[df['GeographicalName']==l_match]['Province'].values[0]):
                            print("****************************ERROR********************")
                            twt['City'][index]= "dummy" #the found city is wrong, remove the found one!
                        else:
                            twt['Province'][index]=df[df['GeographicalName']==l_match]['Province'].values[0] 
                            print(df[df['GeographicalName']==l_match]['Province'].values[0])
                            twt['City'][index]= df[df['GeographicalName']==l_match]['GeographicalName'].values[0]
                else:
                    twt['Province'][index]=df[df['GeographicalName']==l_match]['Province'].values[0] 
                    print(df[df['GeographicalName']==l_match]['Province'].values[0])
                    twt['City'][index]= df[df['GeographicalName']==l_match]['GeographicalName'].values[0]
                

In [ ]:
#Canada.csv==the geographical dataset (can be changed to other dictionaries)

canada_loc = pd.read_csv("Canada.csv", sep=',',header=0)
area = ['City', 'Regional Municipality', 'Town', 'District Municipality', 'Township Municipality', 'Municipal District',
       'Dispersed Rural Community', 'Indian Reserve','Village','Municipality', 'Regional District', 'country', 'convert',
        'Resort Municipality', 'Separated Town', 'Rural Municipality','County Regional Municipality', 'Hill', 'Community','Recreational Community']

stop_words = ['Trail', 'Park', 'Street', 'Central', 'Centre', 'Center', 'Black', 'Boys', 'Home', 'Lake', 'Shaw', 'Hall', 'David', 'West', 'Little']

#filter the dataset to cities to exclude all the tweets that are explicitly mapped to a city
#df= canada_loc.loc[canada_loc['Generic Term'].isin(area)] 
df=canada_loc
#df= canada_loc
ct = ['City', 'Town']
c = ['City']
df_cities= canada_loc.loc[canada_loc['Generic Term'].isin(ct)] 
df_only_cities= canada_loc.loc[canada_loc['Generic Term'].isin(c)] 
cities=df_cities['GeographicalName'].unique() #remove this if the line below worked Ok!
#cities=df['GeographicalName'].unique()
cities_only=df_only_cities['GeographicalName'].unique()

#--------------------------------------------------------------




#need to read the csv file in this way as it had lots of line breaks in some of the columns
with open("inputRawdata.csv") as f:
    reader = csv.reader(f)
    headers = next(reader, None)
    csv_rows = list(reader)
headers


twt = pd.DataFrame(csv_rows, columns=headers)
twt['City']="dummy"
twt['Province']="dummy"
find_city(twt)
find_dummy_province(twt)
find_dummy(twt)
df= canada_loc.loc[~canada_loc['GeographicalName'].isin(stop_words)] 
final_find(twt)
find_from_user(twt)
find_from_text(twt)

In [ ]:
twt=twt.drop_duplicates(subset='id_str', keep="last")

In [ ]:
twt.to_csv("OutputLocation.csv", sep=',', encoding='utf-8', index=False)